In [1]:
# a. Data Preparation
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.layers import Embedding, Dot, Reshape, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity

# Sample text corpus (you can replace this with any corpus)
corpus = [
    "the quick brown fox jumps over the lazy dog",
    "the dog is in the garden",
    "the fox is in the forest"
]

# Tokenize the corpus
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}  # Reverse mapping for easy access

# Convert sentences to sequences of word indices
sequences = tokenizer.texts_to_sequences(corpus)

# b. Generate Training Data
# Generate context-target pairs for CBOW
window_size = 2  # Number of context words to use on each side of the target word

def generate_context_target_pairs(sequences, window_size):
    context_target_pairs = []
    for sequence in sequences:
        for i, target_word in enumerate(sequence):
            context_words = [
                sequence[j] for j in range(max(0, i - window_size), min(len(sequence), i + window_size + 1))
                if j != i
            ]
            context_target_pairs.append((context_words, target_word))
    return context_target_pairs

context_target_pairs = generate_context_target_pairs(sequences, window_size)

# Prepare the training data for the CBOW model
def prepare_training_data(context_target_pairs, vocab_size):
    X_context = []
    y_target = []
    for context, target in context_target_pairs:
        context_vector = pad_sequences([context], maxlen=2 * window_size)  # Pad to ensure fixed input size
        X_context.append(context_vector.flatten())
        y_target.append(target)
    return np.array(X_context), np.array(y_target)

X_train, y_train = prepare_training_data(context_target_pairs, vocab_size)

# c. Train Model
embedding_dim = 50  # Dimension of the word embeddings

# Define the CBOW model
input_context = Input(shape=(2 * window_size,))
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=2 * window_size)(input_context)
average_embedding = tf.reduce_mean(embedding, axis=1)
output = Dense(vocab_size, activation='softmax')(average_embedding)

cbow_model = Model(inputs=input_context, outputs=output)
cbow_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
cbow_model.fit(X_train, y_train, epochs=100, verbose=1)

# d. Output: Get Word Embeddings and Similarity
# Extract the word embeddings from the model
word_embeddings = cbow_model.get_layer('embedding').get_weights()[0]

# Function to find similar words
def find_similar_words(word, word_embeddings, word_index, index_word, top_n=5):
    word_id = word_index[word]
    word_vector = word_embeddings[word_id].reshape(1, -1)
    similarities = cosine_similarity(word_vector, word_embeddings)
    similar_word_ids = similarities[0].argsort()[-top_n-1:-1][::-1]
    similar_words = [index_word[i] for i in similar_word_ids if i != word_id]
    return similar_words

# Example: Find words similar to "fox"
similar_words = find_similar_words("fox", word_embeddings, word_index, index_word)
print(f"Words similar to 'fox': {similar_words}")

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 2.5663
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 2.5626
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 2.5589
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 2.5552
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 2.5515
Epoch 6/100
1/1 [==============================] - 0s 16ms/step - loss: 2.5478
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 2.5441
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 2.5405
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 2.5368
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: 2.5331
Epoch 11/100
1/1 [==============================] - 0s 11ms/step - loss: 2.5294
Epoch 12/100
1/1 [==============================] - 0s 16ms/step - loss: 2.5258
Epoch 13/100
1/1 [==============================] - 